### Chatbot with multiple tools

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

google_api = os.environ['GOOGLE_API_KEY']


In [3]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

In [4]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=3, doc_content_chars_max=700)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [5]:
arxiv.invoke("Agentic AI")

"Published: 2025-07-02\nTitle: AI Agents and Agentic AI-Navigating a Plethora of Concepts for Future Manufacturing\nAuthors: Yinwang Ren, Yangyang Liu, Tang Ji, Xun Xu\nSummary: AI agents are autonomous systems designed to perceive, reason, and act within\ndynamic environments. With the rapid advancements in generative AI (GenAI),\nlarge language models (LLMs) and multimodal large language models (MLLMs) have\nsignificantly improved AI agents' capabilities in semantic comprehension,\ncomplex reasoning, and autonomous decision-making. At the same time, the rise\nof Agentic AI highlights adaptability and goal-directed autonomy in dynamic and\ncomplex environments. LLMs-based AI Agents (LLM-Agents), MLLM"

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily = TavilySearchResults()

tavily.invoke("Who is 2nd highest run getter in test cricket")

[{'title': 'Joe Root: England batting star becomes second highest ... - Sky Sports',
  'url': 'https://www.skysports.com/cricket/news/12123/13401678/joe-root-england-batting-star-becomes-second-highest-all-time-test-run-scorer-behind-only-sachin-tendulkar',
  'content': "Joe Root now sits second on the list of all-time Test run-scorers - and there is a more than decent chance he ends up top of the pile before his career is over.\n\nThe England batter pushed past Rahul Dravid, Jacques Kallis and Ricky Ponting while scoring 150 - his 38th Test ton - on day three of the fourth Test against India at Emirates Old Trafford and it is now only Sachin Tendulkar keeping him off the summit. [...] Sky Sports\n\nCricket\n\n#### Internationals\n\n#### Domestic\n\n#### Indian Premier League\n\n#### Domestic Leagues\n\n#### Internationals\n\n# Joe Root: England batting star becomes second highest all-time Test run-scorer behind only Sachin Tendulkar [...] Joe Root moves past Rahul Dravid, Jacques Kall

In [9]:
tools = [arxiv, tavily]

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, google_api_key=google_api)

d:\Ineuron-materials-FSDS\Gen AI\Learning-langgraph\langgraph\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
llm_with_tools = llm.bind_tools(tools)

from pprint import pprint
from langchain_core.messages import HumanMessage

llm_with_tools.invoke([HumanMessage(content="Who 2nd Highest run getter in ODI cricket")])

AIMessage(content='I am sorry, I cannot answer this question. I do not have access to real-time information, including sports statistics.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7e8e4842-9388-4fe7-adcf-1eb1b64ec64d-0', usage_metadata={'input_tokens': 119, 'output_tokens': 26, 'total_tokens': 145, 'input_token_details': {'cache_read': 0}})